# DSO105 Intermediate Stats L3 Hands On - R

In [1]:
## load libraries

library(gmodels)
library(tidyverse)

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.1.2     v dplyr   1.0.6
v tidyr   1.1.3     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



In [2]:
## load data

loans = read.csv('../../datasets/loans.csv')

In [3]:
head(loans)

,X,term,loan_status,Date,home_ownership
,<int>,<chr>,<chr>,<chr>,<chr>
1,1,36 months,Fully Paid,12/1/2011,RENT
2,2,60 months,Charged Off,12/1/2011,RENT
3,3,36 months,Fully Paid,12/1/2011,RENT
4,4,36 months,Fully Paid,12/1/2011,RENT
5,5,60 months,Current,12/1/2011,RENT
6,6,36 months,Fully Paid,12/1/2011,RENT


For each part, check and correct for assumptions if possible, perform the appropriate categorical data analysis in R, 

and provide a one-sentence conclusion at the bottom of your program files about the analysis you performed.

### Does the term of the loan influence loan status? If so, how?

In [12]:
CrossTable(loans$term, loans$loan_status, fisher=TRUE, chisq = TRUE, expected = TRUE, sresid=TRUE, format="SPSS")


   Cell Contents
|-------------------------|
|                   Count |
|         Expected Values |
| Chi-square contribution |
|             Row Percent |
|          Column Percent |
|           Total Percent |
|            Std Residual |
|-------------------------|

Total Observations in Table:  21957 

             | loans$loan_status 
  loans$term | Charged Off  |     Current  |  Fully Paid  |   Row Total | 
-------------|-------------|-------------|-------------|-------------|
   36 months |       2029  |          0  |      14964  |      16993  | 
             |   2540.011  |    388.509  |  14064.480  |             | 
             |    102.808  |    388.509  |     57.530  |             | 
             |     11.940% |      0.000% |     88.060% |     77.392% | 
             |     61.822% |      0.000% |     82.342% |             | 
             |      9.241% |      0.000% |     68.151% |             | 
             |    -10.139  |    -19.711  |      7.585  |             | 
-------

ERROR: Error in fisher.test(t, alternative = "two.sided"): FEXACT error 6 (f5xact).  LDKEY=484 is too small for this problem: kval=84285355.
Try increasing the size of the workspace.


In [13]:
## p-value is significant. Loan term does influence loan status.
## 36 month terms look more likely to be paid back

### How has the ability to own a home changed from 2007 to 2011?

In [27]:
loans1 <- separate(loans, Date, c("Ignore", "Issue_Month", "Issue_Year"), sep="/")

In [28]:
head(loans1)

,X,term,loan_status,Ignore,Issue_Month,Issue_Year,home_ownership
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,36 months,Fully Paid,12,1,2011,RENT
2,2,60 months,Charged Off,12,1,2011,RENT
3,3,36 months,Fully Paid,12,1,2011,RENT
4,4,36 months,Fully Paid,12,1,2011,RENT
5,5,60 months,Current,12,1,2011,RENT
6,6,36 months,Fully Paid,12,1,2011,RENT


In [40]:
loans1$Issue_YearR <- NA
loans1$Issue_YearR[loans1$Issue_Year == "2007"] <- 0
loans1$Issue_YearR[loans1$Issue_Year == "2011"] <- 1

In [30]:
loans1$RentvOwn <- NA
loans1$RentvOwn[loans1$home_ownership == "RENT"] <- 0
loans1$RentvOwn[loans1$home_ownership == "OWN"] <- 1

In [41]:
head(loans1)

,X,term,loan_status,Ignore,Issue_Month,Issue_Year,home_ownership,Issue_MonthR,RentvOwn,Issue_YearR
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,1,36 months,Fully Paid,12,1,2011,RENT,1,0,1
2,2,60 months,Charged Off,12,1,2011,RENT,1,0,1
3,3,36 months,Fully Paid,12,1,2011,RENT,1,0,1
4,4,36 months,Fully Paid,12,1,2011,RENT,1,0,1
5,5,60 months,Current,12,1,2011,RENT,1,0,1
6,6,36 months,Fully Paid,12,1,2011,RENT,1,0,1


In [31]:
## use McNemar Chi-Square bc we are working with 2 points in time and looking for the change between them

In [42]:
CrossTable(loans1$RentvOwn, loans1$Issue_MonthR, fisher=TRUE, chisq = TRUE, mcnemar = TRUE, expected = TRUE, sresid=TRUE, format="SPSS")


   Cell Contents
|-------------------------|
|                   Count |
|         Expected Values |
| Chi-square contribution |
|             Row Percent |
|          Column Percent |
|           Total Percent |
|            Std Residual |
|-------------------------|

Total Observations in Table:  11692 

                | loans1$Issue_MonthR 
loans1$RentvOwn |        0  |        1  | Row Total | 
----------------|-----------|-----------|-----------|
              0 |      130  |     9929  |    10059  | 
                |  130.770  | 9928.230  |           | 
                |    0.005  |    0.000  |           | 
                |    1.292% |   98.708% |   86.033% | 
                |   85.526% |   86.040% |           | 
                |    1.112% |   84.921% |           | 
                |   -0.067  |    0.008  |           | 
----------------|-----------|-----------|-----------|
              1 |       22  |     1611  |     1633  | 
                |   21.230  | 1611.770  |        

In [44]:
## p-value says there is a difference
## standardized residuals shows that ratio of rent:own is similar from 2007 to 2011

The news just ran a story that only 15% of homes are fully paid for in America, and that another 10% have given up on paying it back, so the bank has "charged off" the loan. Does it seem likely that the data for this hands on came from the larger population of America?

### Goodness of Fit Chi-Square

In [46]:
loans %>% group_by(loan_status) %>% summarize(count=n())

loan_status,count
<chr>,<int>
Charged Off,3282
Current,502
Fully Paid,18173


In [47]:
observed = c(3382, 502, 18173)

In [48]:
expected = c(0.1,0.75,0.15)

In [49]:
chisq.test(x = observed, p = expected)


	Chi-squared test for given probabilities

data:  observed
X-squared = 82963, df = 2, p-value < 2.2e-16


In [ ]:
## the p-value shows that there is significant difference between our sample and population